## Open file and select text column

In [65]:
!pip install nltk
!pip install textblob

In [66]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import Tokenizer, StopWordsRemover
import pandas as p
import nltk
from textblob import TextBlob

# Initialize the Spark session, no configurations?
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("Assignment2") \
    .getOrCreate()

# Read the table in csv format
# tweet_table = spark.read.format("bigquery").load("de2021labs.twitter_data.twitter_data")

# Setup hadoop fs configuration
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl","com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl","com.google.cloud.hadoop.fs.gcs.GoogleHadoopF5")

# Google storage filepath
gsc_file_path = 'gs://elise_ass2_input/vaccination_tweets.csv'
tweet_table = (spark.read
      .option("multiline", "true")
      .option("quote", '"')
      .option("header", "true")
      .option("escape", "\\")
      .option("escape", '"')
      .csv(gsc_file_path)
)

# Print schema and number of tweets in the dataset
print("Table schema: {}\n".format(tweet_table.schema))
print("Number of tweets: {}\n".format(tweet_table.count()))

# Select tweet text and peak into the text
tweet_text_table = tweet_table.select(['text','date'])
tweet_text_table.show(10,truncate=False)

,StringType,true)))tType(List(StructField(id,StringType,true),StructField(user_name,StringType,true),StructField(user_location,StringType,true),StructField(user_description,StringType,true),StructField(user_created,StringType,true),StructField(user_followers,StringType,true),StructField(user_friends,StringType,true),StructField(user_favourites,StringType,true),StructField(user_verified,StringType,true),StructField(date,StringType,true),StructField(text,StringType,true),StructField(hashtags,StringType,true),StructField(source,StringType,true),StructField(retweets,StringType,true),StructField(favorites,StringType,true),StructField(is_retweet

Number of tweets: 8082

+--------------------------------------------------------------------------------------------------------------------------------------------+-------------------+
|text                                                                                                                                        |date               |
+

### Cleaning

In [67]:
# Removing empty tweets
tweet_text_table = tweet_text_table.na.drop()
print("Number of tweets after removing null: {}\n".format(tweet_text_table.count()))

# Remove twitter handlers, hashtags, URLS, special characters, single characters and double spaces respectively
df_clean = tweet_text_table.select('text', ((regexp_replace('text', '@[^\s]+', "")).alias('text2')),'date')
df_clean = df_clean.select('text2', ((regexp_replace('text2', r'\B#\S+', "")).alias('text3')),'date')
df_clean = df_clean.select('text3', ((regexp_replace('text3', r"http\S+", "")).alias('text4')),'date')
df_clean = df_clean.select('text4', ((regexp_replace('text4', r'[^\w+]', " ")).alias('text5')),'date') 
df_clean = df_clean.select('text5', ((regexp_replace('text5', r'\s+[a-zA-Z]\s+', " ")).alias('text6')),'date')
df_clean = df_clean.select('text6', ((regexp_replace('text6', r'\s+', " ")).alias('text')),'date').select(['text','date'])

# Create index
df_clean = df_clean.select("*") \
                    .withColumn("id", monotonically_increasing_id()) \
                    .select("id","text",'date')

# Peak
df_clean.show(10,truncate=False)

Number of tweets after removing null: 8082

+---+------------------------------------------------------------------------------------------------------------------+-------------------+
|id |text                                                                                                              |date               |
+---+------------------------------------------------------------------------------------------------------------------+-------------------+
|0  |Same folks said daikon paste could treat cytokine storm                                                           |2020-12-20 06:06:44|
|1  |While the world has been on the wrong side of history this year hopefully the biggest vaccination effort we ve ev |2020-12-13 16:27:13|
|2  | Russian vaccine is created to last 2 4 years                                                                     |2020-12-12 20:33:45|
|3  |Facts are immutable Senator even when you re not ethically sturdy enough to acknowledge them 1 You were b

## Sentiment analysis

In [68]:
# There was an issue with the texblob module such that it did not allow for application of an udf to an entire column. 
# To fix this, we converted the pyspark df to a pandas df and simply appended a list of sentiments to the df.

pandas_df = df_clean.toPandas()
sentiment_list = list()

for index, row in pandas_df.iterrows():
    sentiment = row['text']
    sentiment_list.append(TextBlob(sentiment).sentiment[0])

sentiments_df = spark.createDataFrame(sentiment_list, FloatType()) \
                    .select("*") \
                    .withColumn("id", monotonically_increasing_id()) 

# Join dataframes
tweet_sentiments_df = df_clean.join(sentiments_df, "id") \
                    .withColumnRenamed("value","sentiment")

# Peak
tweet_sentiments_df.show(10, truncate=False)


+---+------------------------------------------------------------------------------------------------------------------+-------------------+---------+
|id |text                                                                                                              |date               |sentiment|
+---+------------------------------------------------------------------------------------------------------------------+-------------------+---------+
|0  |Same folks said daikon paste could treat cytokine storm                                                           |2020-12-20 06:06:44|0.0      |
|1  |While the world has been on the wrong side of history this year hopefully the biggest vaccination effort we ve ev |2020-12-13 16:27:13|-0.5     |
|2  | Russian vaccine is created to last 2 4 years                                                                     |2020-12-12 20:33:45|0.0      |
|3  |Facts are immutable Senator even when you re not ethically sturdy enough to acknowledge t

## Grouping and binning results

In [74]:
# Grouping the average sentiments by month and year to get a better and more consise, aggregated overview

twitter_grouped_sentiments = tweet_sentiments_df.groupBy(to_date("date")) \
                                .agg(avg("sentiment"), count("*"))

# Creating bins, here 0 is neutral, <0 is negative and >0 is positive. This will help visualisation.
def categorizer(s):
    if s == -1:
        return "-1"
    elif -1 < s <= -0.5:
        return "-0.75"
    elif -0.5 < s < 0:
        return "-0.25"
    elif s == 0:
        return "0"
    elif 0 < s <0.5:
        return "0.25"
    elif 0.5<= s <1:
        return "0.75"
    else:
        return "1"
        
bin_udf = udf(categorizer, StringType() )
bin_df = twitter_grouped_sentiments.withColumn("bin", bin_udf("avg(sentiment)"))
twitter_final_sentiment_df = bin_df.select("*") \
                                .withColumnRenamed("to_date(date)","date") \
                                .withColumnRenamed("month(timestamp)","month") \
                                .withColumnRenamed("count(1)","record_count") \
                                .withColumnRenamed("avg(sentiment)","average_sentiment") 
twitter_final_sentiment_df.show()

+----------+--------------------+------------+-----+
|      date|   average_sentiment|record_count|  bin|
+----------+--------------------+------------+-----+
|2020-12-18|  0.1329552700127741|         112| 0.25|
|2020-12-17| 0.12265047720439143|         139| 0.25|
|2021-01-18| 0.06291308657223867|          66| 0.25|
|2021-01-25|-0.02111111208796501|           5|-0.25|
|2020-12-25| 0.11345012737438083|          40| 0.25|
|2020-12-22|  0.1341232542648532|         157| 0.25|
|2020-12-30| 0.15066185927836695|         107| 0.25|
|2020-12-13| 0.09941434708176827|          98| 0.25|
|2020-12-23| 0.17029390916061715|         123| 0.25|
|2020-12-21|  0.1297358812344279|         102| 0.25|
|2021-01-05| 0.15419292654674333|         111| 0.25|
|2020-12-12| 0.07747474797661343|          63| 0.25|
|2020-12-16| 0.10071892046188521|         144| 0.25|
|2021-01-19| 0.08660640548491343|          88| 0.25|
|2020-12-24| 0.17471763400949145|         114| 0.25|
|2020-12-20| 0.10532212818917983|          68|

The next step is to save the sentiment analysis df to BigQuery.

In [ ]:
# Use the Cloud Storage bucket for temporary BigQuery export 
bucket = "elise_ass2_temp"
spark.conf.set('temporaryGcsBucket', bucket)

# Saving the data to BigQuery
twitter_final_sentiment_df.write.format('bigquery') \
  .option('table', 'nodal-strength-325610.assignment2.twitter_sentiment_v2') \
  .mode("overwrite") \
  .save()


## Word count

In [ ]:
df_clean.select("text").show(10)

In [ ]:
import pyspark.sql.functions as f
from pyspark.ml.feature import Tokenizer, StopWordsRemover

# Tokenize
tokenizer = Tokenizer(inputCol='text', outputCol='body_tokenized')
twitter_data_text_tokenized = tokenizer.transform(df_clean).select('id', 'body_tokenized')

# Remove unnecessary words
unncessary_words_list = ["like", "know", "get", 'one', 'think', 'cause', 'say', 'even', "don't", 'got', 'also', 'good', 'said',
                        'make', 'it.', 'first', 'many', 'still', 'actually', "don't", 'want', 'read', 'print', 'vaccine',
                        'vaccines', 'vaccinated', 'vaccination', 'may', 'saying', 'point', 'virus', 'never', 'much', 'see',
                        '1', 'way', 'wrong', 'really', 'used', 'well', 'getting', 'take', 'every', 'go', '>'] 
unncessary_words_list.extend(StopWordsRemover().getStopWords())
remover = StopWordsRemover(inputCol='body_tokenized', outputCol='body_clean', stopWords=unncessary_words_list)
twitter_data_text_no_stopwords = remover.transform(twitter_data_text_tokenized).select(['id', 'body_clean'])

# Return to regular strings
twitter_data_text_no_stopwords = twitter_data_text_no_stopwords.withColumn("body_clean", 
                                                                         concat_ws(",", "body_clean"))

# Count the words
count_df = twitter_data_text_no_stopwords.withColumn('body_clean', f.explode(f.split(f.col('body_clean'), ',')))\
    .groupBy('body_clean')\
    .count()\
    .sort('count', ascending=False)\
    .filter(f.col('body_clean') != "")\
    .withColumnRenamed("body_clean","word")

count_df.show()

In [ ]:
# Save the word count
count_df.write.format('bigquery') \
  .option('table', 'nodal-strength-325610.assignment2.twitter_word_count') \
  .mode("overwrite") \
  .save()

At last, the spark context should be stopped.

In [ ]:
spark.stop()